# Análisis de Accidentes de Tráfico en España (2023)

Este notebook procesa y analiza datos de accidentes de tráfico en España durante el año 2023, combinándolos con información del parque de vehículos por provincia.

El proceso se divide en tres partes:
1. Limpieza y procesamiento de datos de accidentes
2. Procesamiento de datos del parque de vehículos
3. Unión de ambos conjuntos de datos

Los datos provienen de fuentes oficiales:
- Accidentes: DGT (Dirección General de Tráfico)
- Parque de vehículos: Estadísticas oficiales del parque automovilístico

In [44]:
from pathlib import Path
import pandas as pd

# Desde esta url cogemos los datos de accidentes de trafico en España y el diccionario de los datos
# URL oficial DGT 2023 (XLSX)
# https://www.dgt.es/menusecundario/dgt-en-cifras/dgt-en-cifras-resultados/dgt-en-cifras-detalle/Ficheros-microdatos-de-accidentes-con-victimas-2023/

# Procesamiento de Datos de Accidentes

En esta sección procesamos los datos de accidentes de tráfico. Los pasos principales son:
- Carga de datos desde archivo Excel
- Limpieza de nombres de columnas
- Conversión de códigos a etiquetas descriptivas
- Creación de variables temporales (tramos horarios)
- Selección de variables relevantes

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

# Rutas 
BASE = Path("..")
RAW  = BASE / "data" / "raw"
PROC = BASE / "data" / "processed"
PROC.mkdir(parents=True, exist_ok=True)

ACC_RAW = RAW / "accidentes" / "TABLA_ACCIDENTES_23.XLSX"
DICC_XLS = RAW / "accidentes" / "Diccionario-Tabla-Accidente.xlsx"

def snake(s: str) -> str:
    return (s.strip()
            .replace("Á","A").replace("É","E").replace("Í","I").replace("Ó","O").replace("Ú","U").replace("Ñ","N")
            .replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u").replace("ñ","n")
            .replace("·"," ").replace("."," ").replace("-","_").replace("/","_")
            .replace("("," ").replace(")"," ").replace("%","pct").replace("+","_")
            .replace("__","_").lower())

def read_dict_sheet(xls_path: Path, sheet_name: str) -> pd.DataFrame:
    """Lee hoja con cabecera 'Valor'/'Etiqueta' y devuelve columnas: codigo(Int64), label(str)."""
    tmp = pd.read_excel(xls_path, sheet_name=sheet_name, header=None)
    # localiza la fila de cabecera
    header_row = None
    for i in range(len(tmp)):
        c0 = str(tmp.iloc[i,0]).strip().lower()
        c1 = str(tmp.iloc[i,1]).strip().lower() if tmp.shape[1] > 1 else ""
        if c0 == "valor" and c1 == "etiqueta":
            header_row = i
            break
    df = pd.read_excel(xls_path, sheet_name=sheet_name, header=(header_row if header_row is not None else 1))
    df.columns = [str(c).strip() for c in df.columns]
    col_code  = next((c for c in df.columns if c.strip().lower() in ("valor","codigo","código","cod","id")), None)
    col_label = next((c for c in df.columns if c.strip().lower() in ("etiqueta","label","nombre","descripcion","descripción")), None)
    out = df[[col_code, col_label]].dropna().copy()
    out.columns = ["codigo","label"]
    out["codigo"] = pd.to_numeric(out["codigo"], errors="coerce").astype("Int64")
    out["label"]  = out["label"].astype(str).str.strip()
    return out

def add_label(df: pd.DataFrame, code_col: str, sheet: str, new_label_col: str) -> pd.DataFrame:
    """Crea <new_label_col> mapeando df[code_col] con la hoja del diccionario <sheet>.
       Mantiene el code_col en el DF (no se borra)."""
    if code_col not in df.columns:
        return df
    mapa = read_dict_sheet(DICC_XLS, sheet)
    out = df.copy()
    out[code_col] = pd.to_numeric(out[code_col], errors="coerce").astype("Int64")
    out = out.merge(mapa, left_on=code_col, right_on="codigo", how="left")
    out[new_label_col] = out["label"].fillna("desconocido")
    out = out.drop(columns=["codigo","label"])
    return out

# Cargar accidentes y normalizar encabezados
acc_raw = pd.read_excel(ACC_RAW, dtype=str)
acc_raw.columns = [snake(c) for c in acc_raw.columns]

# Selección + renombrado claro
cmap = {
    # TIEMPO (queremos códigos y luego labels auxiliares)
    "anyo":"anio", "mes":"mes", "dia_semana":"dia_semana", "hora":"hora",
    # TERRITORIO (queremos códigos; añadiremos provincia_label)
    "cod_provincia":"cod_provincia", "cod_municipio":"cod_municipio",
    # VIA / ZONA (están codificados → los mapeamos a labels)
    "zona":"zona_cod", "zona_agrupada":"zona_agrupada_cod",
    "carretera":"carretera", "sentido_1f":"sentido_cod",
    "titularidad_via":"titularidad_via_cod", "tipo_via":"tipo_via_cod",
    # ACCIDENTE / SEVERIDAD (métricas a 30 días)
    "tipo_accidente":"tipo_accidente_cod",
    "total_mu30df":"muertos_30d", "total_hg30df":"heridos_graves_30d",
    "total_hl30df":"heridos_leves_30d", "total_victimas_30df":"victimas_30d",
    "total_vehiculos":"total_vehiculos",
    # USUARIOS (30d)
    "tot_peat_mu30df":"peatones_muertos_30d",
    "tot_moto_mu30df":"motos_muertos_30d",
    "tot_bici_mu30df":"bicis_muertos_30d",
    "tot_tur_mu30df":"turismos_muertos_30d",
    "tot_vmp_mu30df":"vmp_muertos_30d",
    # CONDICIONES (codificadas)
    "condicion_iluminacion":"cond_ilum_cod",
    "condicion_meteo":"cond_meteo_cod",
    "condicion_firme":"cond_firme_cod",
    "visib_restringida_por":"visib_rest_cod",
}
keep = {k:v for k,v in cmap.items() if k in acc_raw.columns}
acc = acc_raw[list(keep.keys())].rename(columns=keep).copy()

# Tipos numéricos donde toca 
num_cols = ["anio","mes","dia_semana","hora","cod_provincia","cod_municipio",
            "muertos_30d","heridos_graves_30d","heridos_leves_30d","victimas_30d","total_vehiculos",
            "peatones_muertos_30d","motos_muertos_30d","bicis_muertos_30d","turismos_muertos_30d","vmp_muertos_30d"]
for c in num_cols:
    if c in acc.columns:
        acc[c] = pd.to_numeric(acc[c], errors="coerce")

# Añadir labels de tiempo y provincia (manteniendo códigos)
mes_map = {1:"ene",2:"feb",3:"mar",4:"abr",5:"may",6:"jun",7:"jul",8:"ago",9:"sep",10:"oct",11:"nov",12:"dic"}
dia_map = {1:"lunes",2:"martes",3:"miercoles",4:"jueves",5:"viernes",6:"sabado",7:"domingo"}  # ajusta si 1=domingo

if "mes" in acc.columns: acc["mes_label"] = acc["mes"].map(mes_map)
if "dia_semana" in acc.columns: acc["dia_semana_label"] = acc["dia_semana"].map(dia_map)

# Provincia label desde el diccionario
acc = add_label(acc, "cod_provincia", "COD_PROVINCIA", "provincia_label")

# Mapear códigos → labels para variables codificadas (conservando los *_cod) 
acc = add_label(acc, "zona_cod",             "ZONA",                     "zona_label")
acc = add_label(acc, "zona_agrupada_cod",    "ZONA_AGRUPADA",            "zona_agrupada_label")
acc = add_label(acc, "sentido_cod",          "SENTIDO_1F",               "sentido_label")
acc = add_label(acc, "titularidad_via_cod",  "TITULARIDAD_VIA",          "titularidad_via_label")
acc = add_label(acc, "tipo_via_cod",         "TIPO_VIA",                 "tipo_via_label")
acc = add_label(acc, "tipo_accidente_cod",   "TIPO_ACCIDENTE",           "tipo_accidente_label")
acc = add_label(acc, "cond_ilum_cod",        "CONDICION_ILUMINACION",    "condicion_iluminacion_label")
acc = add_label(acc, "cond_meteo_cod",       "CONDICION_METEO",          "condicion_meteo_label")
acc = add_label(acc, "cond_firme_cod",       "CONDICION_FIRME",          "condicion_firme_label")
acc = add_label(acc, "visib_rest_cod",       "VISIB_RESTRINGIDA_POR",    "visib_restringida_por_label")

# tramo horario legible (mantenemos 'hora')
if "hora" in acc.columns:
    acc["tramo_horario_label"] = pd.cut(acc["hora"].astype(float),
                                        bins=[-0.1,5,11,14,19,23],
                                        labels=["madrugada","manana","mediodia","tarde","noche"])

# Selección final para GUARDAR (solo labels + métricas + códigos acordados) 
cols_final = [
    # CÓDIGOS que sí conservamos
    "anio","mes","dia_semana","hora","cod_provincia","cod_municipio",
    # Labels de tiempo/territorio
    "mes_label","dia_semana_label","provincia_label","tramo_horario_label",
    # Labels de zona/vía/accidente/condiciones
    "zona_label","zona_agrupada_label","tipo_via_label","sentido_label",
    "titularidad_via_label","tipo_accidente_label",
    "condicion_iluminacion_label","condicion_meteo_label","condicion_firme_label",
    "visib_restringida_por_label",
    # Otros útiles
    "carretera",
    # Métricas numéricas (30 días)
    "muertos_30d","heridos_graves_30d","heridos_leves_30d","victimas_30d","total_vehiculos",
    "peatones_muertos_30d","motos_muertos_30d","bicis_muertos_30d","turismos_muertos_30d","vmp_muertos_30d",
]

cols_final = [c for c in cols_final if c in acc.columns]  
acc_final = acc[cols_final].copy()

# Guardar 
out_path = PROC / "accidentes_clean_2023.csv"
acc_final.to_csv(out_path, index=False)
print("Guardado:", out_path, "| filas:", len(acc_final), "| columnas:", len(acc_final.columns))
acc_final.head(3)


Guardado: ..\data\processed\accidentes_clean_2023.csv | filas: 101306 | columnas: 31


,anio,mes,dia_semana,hora,cod_provincia,cod_municipio,mes_label,dia_semana_label,provincia_label,tramo_horario_label,...,muertos_30d,heridos_graves_30d,heridos_leves_30d,victimas_30d,total_vehiculos,peatones_muertos_30d,motos_muertos_30d,bicis_muertos_30d,turismos_muertos_30d,vmp_muertos_30d
0,2023,1,1,7,1,0,ene,lunes,Araba/Álava,manana,...,0,0,1,1,5,0,0,0,0,0
1,2023,1,1,5,1,1059,ene,lunes,Araba/Álava,madrugada,...,0,0,3,3,2,0,0,0,0,0
2,2023,1,2,9,1,0,ene,martes,Araba/Álava,manana,...,0,0,3,3,2,0,0,0,0,0


In [46]:
# Resumen del dataset de accidentes
print("Dimensiones del dataset:", acc_final.shape)
print("\nPrimeras filas:")
display(acc_final.head(3))

print("\nTipos de datos:")
display(acc_final.dtypes)

print("\nValores nulos por columna:")
nulos = acc_final.isnull().sum()
if nulos.any():
    display(nulos[nulos > 0])
else:
    print("No hay valores nulos en el dataset")

Dimensiones del dataset: (101306, 31)

Primeras filas:


,anio,mes,dia_semana,hora,cod_provincia,cod_municipio,mes_label,dia_semana_label,provincia_label,tramo_horario_label,...,muertos_30d,heridos_graves_30d,heridos_leves_30d,victimas_30d,total_vehiculos,peatones_muertos_30d,motos_muertos_30d,bicis_muertos_30d,turismos_muertos_30d,vmp_muertos_30d
0,2023,1,1,7,1,0,ene,lunes,Araba/Álava,manana,...,0,0,1,1,5,0,0,0,0,0
1,2023,1,1,5,1,1059,ene,lunes,Araba/Álava,madrugada,...,0,0,3,3,2,0,0,0,0,0
2,2023,1,2,9,1,0,ene,martes,Araba/Álava,manana,...,0,0,3,3,2,0,0,0,0,0



Tipos de datos:


anio                              int64
mes                               int64
dia_semana                        int64
hora                              int64
cod_provincia                     Int64
cod_municipio                     int64
mes_label                        object
dia_semana_label                 object
provincia_label                  object
tramo_horario_label            category
zona_label                       object
zona_agrupada_label              object
tipo_via_label                   object
sentido_label                    object
titularidad_via_label            object
tipo_accidente_label             object
condicion_iluminacion_label      object
condicion_meteo_label            object
condicion_firme_label            object
visib_restringida_por_label      object
carretera                        object
muertos_30d                       int64
heridos_graves_30d                int64
heridos_leves_30d                 int64
victimas_30d                      int64



Valores nulos por columna:
No hay valores nulos en el dataset


# Procesamiento del Parque de Vehículos

En esta sección procesamos los datos del parque de vehículos por provincia. Los pasos son:
- Carga del archivo Excel y detección automática de la hoja correcta
- Limpieza y selección de columnas relevantes
- Mapeo de provincias a códigos INE
- Cálculo de porcentajes por tipo de vehículo

In [ ]:
# Desde esta url cogemos los datos del parque de vehículos por provincia
# https://www.dgt.es/menusecundario/dgt-en-cifras/dgt-en-cifras-resultados/dgt-en-cifras-detalle/Parque-de-vehiculos-Tablas-Estadisticas-2023/

# --- Rutas del proyecto ---
BASE = Path("..")
RAW  = BASE / "data" / "raw"
PROC = BASE / "data" / "processed"
PROC.mkdir(parents=True, exist_ok=True)

PARK_XLS = RAW / "parque" / "Parque-de-vehiculos-Tablas-estadisticas-2023.xlsx"

In [ ]:
# Listar hojas y elegir la que contiene la tabla provincial
xls = pd.ExcelFile(PARK_XLS)
candidate = None
for sh in xls.sheet_names:
    if "v_4" in sh.lower():
        candidate = sh
        break
print("Hoja detectada:", candidate)

# Detectar fila de cabecera (busca una fila con 'Turismos'/'Motocicletas' y 'Provincias')
tmp = pd.read_excel(PARK_XLS, sheet_name=candidate, header=None)
header_row = None
for i in range(min(40, len(tmp))):
    rowtxt = " ".join([str(x) for x in tmp.iloc[i].tolist() if pd.notna(x)]).lower()
    if ("turism" in rowtxt or "motocic" in rowtxt) and ("provin" in rowtxt or "camiones" in rowtxt):
        header_row = i
        break
assert header_row is not None, "No se encontró cabecera; revisa la hoja del Excel."

raw = pd.read_excel(PARK_XLS, sheet_name=candidate, header=header_row)

# Antes de aplicar snake_case, guardamos los nombres originales
nombres_originales = list(raw.columns)
raw.columns = [snake(c) for c in raw.columns]

# Detecta la columna de provincias y las columnas de vehículos
col_prov = next((c for c in raw.columns if c.startswith("provinc")), None)

# Mapeo de las columnas que queremos con sus posibles variantes
columnas_mapping = {
    "camiones": ["camiones"],
    "furgonetas": ["furgonetas"],
    "autobuses": ["autobuses"],
    "turismos": ["turismos"],
    "motocicletas": ["motocicletas"],
    "total": ["total"],
    "vehiculos_por_1000": ["vehiculos por 1 000", "vehiculos_por_1000", "vehiculos por 1000"]
}

# Seleccionar columnas existentes
veh_cols = []
for col_desired, variants in columnas_mapping.items():
    # Buscar la columna en cualquiera de sus variantes
    found_col = None
    for variant in variants:
        matching_cols = [c for c in raw.columns if variant in c.lower()]
        if matching_cols:
            found_col = matching_cols[0]
            break
    if found_col:
        veh_cols.append(found_col)

# Crear DataFrame final con las columnas seleccionadas
cols_keep = [col_prov] + veh_cols
df = raw[cols_keep].copy()

# Limpiar filas vacías/notas/totales
df = df[~df[col_prov].isna()]
df = df[df[col_prov].astype(str).str.strip().ne("")]
df = df[~df[col_prov].str.contains("total", case=False, na=False)]

print("Filas post-limpieza:", len(df))
print("Columnas seleccionadas:", df.columns.tolist())

# Eliminar filas con nulos
df = df.dropna()

Hoja detectada: V_4
Filas post-limpieza: 53
Columnas seleccionadas: ['provincias', 'camiones', 'furgonetas', 'autobuses', 'turismos', 'motocicletas', 'total', 'vehiculos por 1 000\nhabitantes*']


c:\Users\alexa\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\Users\alexa\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [ ]:
# Mapear provincia del parque -> cod_provincia (INE 1..52) 

PROV_COL = "provincias"  

prov_to_code = {
    "Araba/Álava": 1,
    "Albacete": 2,
    "Alicante/Ala": 3,
    "Almería": 4,
    "Ávila": 5,
    "Badajoz": 6,
    "Balears (Ill": 7,
    "Barcelona": 8,
    "Burgos": 9,
    "Cáceres": 10,
    "Cádiz": 11,
    "Castellón/C": 12,
    "Ciudad Real": 13,
    "Córdoba": 14,
    "Coruña (A)": 15,
    "Cuenca": 16,
    "Girona": 17,
    "Granada": 18,
    "Guadalajara": 19,
    "Gipuzkoa": 20,
    "Huelva": 21,
    "Huesca": 22,
    "Jaén": 23,
    "León": 24,
    "Lleida": 25,
    "Rioja (La)": 26,
    "Lugo": 27,
    "Madrid": 28,
    "Málaga": 29,
    "Murcia": 30,
    "Navarra": 31,
    "Ourense": 32,
    "Asturias": 33,
    "Palencia": 34,
    "Palmas (Las)": 35,
    "Pontevedra": 36,
    "Salamanca": 37,
    "Santa Cruz d": 38,
    "Cantabria": 39,
    "Segovia": 40,
    "Sevilla": 41,
    "Soria": 42,
    "Tarragona": 43,
    "Teruel": 44,
    "Toledo": 45,
    "Valencia/Val": 46,
    "Valladolid": 47,
    "Bizkaia": 48,
    "Zamora": 49,
    "Zaragoza": 50,
    "Ceuta": 51,
    "Melilla": 52,
}

# Nueva columna con el código INE
df["cod_provincia"] = df[PROV_COL].map(prov_to_code).astype("Int64")

# Comprobación rápida de nulos (1 línea)
print("nulos cod_provincia:", int(df["cod_provincia"].isna().sum()))


nulos cod_provincia: 0


In [50]:
# Renombrar 'total' -> 'vehiculos_totales' si existe
if "total" in df.columns and "vehiculos_totales" not in df.columns:
    df = df.rename(columns={"total": "vehiculos_totales"})
    
# Auditoría básica de la tabla de parque
print("Shape:", df.shape)
print("\nDtypes:")
print(df.dtypes)

print("\nNulos por columna:")
print(df.isna().sum().sort_values(ascending=False))

# Provincias únicas y posibles duplicados
print("\nProvincias únicas:", df["provincias"].nunique())
dup = df["provincias"].duplicated(keep=False).sum()
print("Filas con provincia duplicada:", dup)

# (1 línea) nulos totales
print("\nNulos totales (1 línea):", int(df.isna().sum().sum()))


Shape: (52, 9)

Dtypes:
provincias                           object
camiones                            float64
furgonetas                          float64
autobuses                           float64
turismos                            float64
motocicletas                        float64
vehiculos_totales                   float64
vehiculos por 1 000\nhabitantes*    float64
cod_provincia                         Int64
dtype: object

Nulos por columna:
provincias                          0
camiones                            0
furgonetas                          0
autobuses                           0
turismos                            0
motocicletas                        0
vehiculos_totales                   0
vehiculos por 1 000\nhabitantes*    0
cod_provincia                       0
dtype: int64

Provincias únicas: 52
Filas con provincia duplicada: 0

Nulos totales (1 línea): 0


In [51]:
# Crea porcentajes por tipo sobre el total de vehículos de la provincia (0..100)
base = "vehiculos_totales"
if base in df.columns:
    for c in ["camiones","furgonetas","autobuses","turismos","motocicletas",
              "tractores_industriales","remolques_y_semirremolques","otros_vehiculos"]:
        if c in df.columns:
            df[f"pct_{c}"] = (df[c] / df[base] * 100).where(df[base] != 0)
            
# Vista rápida
pct_cols = [c for c in df.columns if c.startswith("pct_")]
print("Añadidas:", pct_cols)

# Guardar tabla procesada
out_path = PROC / "parque_clean_2023.csv"
df.to_csv(out_path, index=False)
print("Guardado:", out_path, "| filas:", len(df), "| columnas:", len(df.columns))

Añadidas: ['pct_camiones', 'pct_furgonetas', 'pct_autobuses', 'pct_turismos', 'pct_motocicletas']
Guardado: ..\data\processed\parque_clean_2023.csv | filas: 52 | columnas: 14


In [52]:
# Resumen del dataset del parque de vehículos
print("Dimensiones del dataset:", df.shape)
print("\nPrimeras filas:")
display(df.head(3))

print("\nEstadísticas básicas de vehículos por provincia:")
cols_num = ['vehiculos_totales', 'turismos', 'motocicletas', 'camiones']
if all(col in df.columns for col in cols_num):
    display(df[cols_num].describe())

Dimensiones del dataset: (52, 14)

Primeras filas:


,provincias,camiones,furgonetas,autobuses,turismos,motocicletas,vehiculos_totales,vehiculos por 1 000\nhabitantes*,cod_provincia,pct_camiones,pct_furgonetas,pct_autobuses,pct_turismos,pct_motocicletas
0,Araba/Álava,13240.0,15828.0,520.0,161002.0,18739.0,220201.0,654.0,1,6.012688,7.187978,0.236148,73.115926,8.509952
1,Albacete,27280.0,24006.0,436.0,205300.0,31234.0,303571.0,784.0,2,8.986366,7.907870,0.143624,67.628331,10.288862
2,Alicante/Ala,93541.0,88561.0,1814.0,1070509.0,186350.0,1478616.0,758.0,3,6.326254,5.989452,0.122682,72.399392,12.603002



Estadísticas básicas de vehículos por provincia:


,vehiculos_totales,turismos,motocicletas,camiones
count,5.200000e+01,5.200000e+01,52.000000,52.000000
mean,6.937546e+05,4.876268e+05,80054.807692,47442.269231
std,8.924867e+05,6.459851e+05,117394.840131,48883.742003
min,6.336700e+04,4.037100e+04,5766.000000,2139.000000
25%,2.423722e+05,1.757072e+05,20465.250000,19303.750000
50%,4.748060e+05,3.222110e+05,43351.000000,35657.500000
75%,7.462072e+05,5.325168e+05,87282.500000,50021.000000
max,5.444019e+06,4.100548e+06,713033.000000,271935.000000


# Unión de Datos

En esta sección unimos los datos de accidentes con los del parque de vehículos a nivel provincial.
Esto nos permitirá analizar los accidentes en el contexto del parque de vehículos de cada provincia.

In [ ]:
from pathlib import Path
import pandas as pd

# Rutas 
BASE = Path("..")
PROC = BASE / "data" / "processed"

# Cargar accidentes 2023 y parque limpio
acc = pd.read_csv(PROC / "accidentes_clean_2023.csv")
parque = pd.read_csv(PROC / "parque_clean_2023.csv")

# Asegurar tipos para la clave de unión 
acc["cod_provincia"] = pd.to_numeric(acc["cod_provincia"], errors="coerce").astype("Int64")
parque["cod_provincia"] = pd.to_numeric(parque["cod_provincia"], errors="coerce").astype("Int64")

# Evitar colisión de columnas (nos quedamos con provincia_label del DF de accidentes) 
cols_parque_to_add = [c for c in parque.columns if c not in ["provincias"]]

# Unión por provincia (LEFT JOIN; 1 fila accidente + columnas del parque) 
acc_enriched = acc.merge(parque[cols_parque_to_add], on="cod_provincia", how="left")

# Comprobaciones rápidas 
print("Cobertura parque (vehículos totales no nulos):",
      round(acc_enriched.get("vehiculos_totales", acc_enriched.get("parque_total")).notna().mean()*100, 2), "%")
print("Nulos totales post-join:", int(acc_enriched.isna().sum().sum()))

# Guardar resultado 
out_path = PROC / "accidentes_2023_enriquecido.csv"
acc_enriched.to_csv(out_path, index=False)
print("Guardado:", out_path, "| filas:", len(acc_enriched), "| cols:", len(acc_enriched.columns))


Cobertura parque (vehículos totales no nulos): 100.0 %
Nulos totales post-join: 0
Guardado: ..\data\processed\accidentes_2023_enriquecido.csv | filas: 101306 | cols: 43
Guardado: ..\data\processed\accidentes_2023_enriquecido.csv | filas: 101306 | cols: 43


In [54]:
# Resumen del dataset final
print("Dimensiones del dataset final:", acc_enriched.shape)
print("\nColumnas disponibles:")
print(acc_enriched.columns.tolist())

print("\nPrimeras filas del dataset final:")
display(acc_enriched.head(3))

print("\nEstadísticas básicas de accidentes y vehículos por provincia:")
cols_stats = ['muertos_30d', 'heridos_graves_30d', 'heridos_leves_30d', 'vehiculos_totales']
display(acc_enriched[cols_stats].describe())

Dimensiones del dataset final: (101306, 43)

Columnas disponibles:
['anio', 'mes', 'dia_semana', 'hora', 'cod_provincia', 'cod_municipio', 'mes_label', 'dia_semana_label', 'provincia_label', 'tramo_horario_label', 'zona_label', 'zona_agrupada_label', 'tipo_via_label', 'sentido_label', 'titularidad_via_label', 'tipo_accidente_label', 'condicion_iluminacion_label', 'condicion_meteo_label', 'condicion_firme_label', 'visib_restringida_por_label', 'carretera', 'muertos_30d', 'heridos_graves_30d', 'heridos_leves_30d', 'victimas_30d', 'total_vehiculos', 'peatones_muertos_30d', 'motos_muertos_30d', 'bicis_muertos_30d', 'turismos_muertos_30d', 'vmp_muertos_30d', 'camiones', 'furgonetas', 'autobuses', 'turismos', 'motocicletas', 'vehiculos_totales', 'vehiculos por 1 000\nhabitantes*', 'pct_camiones', 'pct_furgonetas', 'pct_autobuses', 'pct_turismos', 'pct_motocicletas']

Primeras filas del dataset final:


,anio,mes,dia_semana,hora,cod_provincia,cod_municipio,mes_label,dia_semana_label,provincia_label,tramo_horario_label,...,autobuses,turismos,motocicletas,vehiculos_totales,vehiculos por 1 000\nhabitantes*,pct_camiones,pct_furgonetas,pct_autobuses,pct_turismos,pct_motocicletas
0,2023,1,1,7,1,0,ene,lunes,Araba/Álava,manana,...,520.0,161002.0,18739.0,220201.0,654.0,6.012688,7.187978,0.236148,73.115926,8.509952
1,2023,1,1,5,1,1059,ene,lunes,Araba/Álava,madrugada,...,520.0,161002.0,18739.0,220201.0,654.0,6.012688,7.187978,0.236148,73.115926,8.509952
2,2023,1,2,9,1,0,ene,martes,Araba/Álava,manana,...,520.0,161002.0,18739.0,220201.0,654.0,6.012688,7.187978,0.236148,73.115926,8.509952



Estadísticas básicas de accidentes y vehículos por provincia:


,muertos_30d,heridos_graves_30d,heridos_leves_30d,vehiculos_totales
count,101306.000000,101306.000000,101306.000000,1.013060e+05
mean,0.017827,0.091456,1.226640,2.030110e+06
std,0.143352,0.321877,0.851598,1.798086e+06
min,0.000000,0.000000,0.000000,6.336700e+04
25%,0.000000,0.000000,1.000000,6.505900e+05
50%,0.000000,0.000000,1.000000,1.179625e+06
75%,0.000000,0.000000,1.000000,3.699208e+06
max,4.000000,7.000000,42.000000,5.444019e+06
